[УТВЕРЖДЕНЫ
постановлением Правительства
Российской Федерации
от 31 октября 2023 г. N 1809](https://ivo.garant.ru/#/document/407923417/paragraph/4/doclist/6925/1/0/0/%D0%9F%D1%80%D0%B5%D0%B4%D0%B5%D0%BB%D1%8C%D0%BD%D0%B0%D1%8F%20%D1%80%D0%B5%D0%BA%D1%80%D0%B5%D0%B0%D1%86%D0%B8%D0%BE%D0%BD%D0%BD%D0%B0%D1%8F%20%D0%B5%D0%BC%D0%BA%D0%BE%D1%81%D1%82%D1%8C%20%D0%B2%20%D0%B3%D1%80%D0%B0%D0%BD%D0%B8%D1%86%D0%B0%D1%85%20%D1%86%D0%B5%D0%BD%D1%82%D1%80%D0%B0%D0%BB%D1%8C%D0%BD%D0%BE%D0%B9%20%D1%8D%D0%BA%D0%BE%D0%BB%D0%BE%D0%B3%D0%B8%D1%87%D0%B5%D1%81%D0%BA%D0%BE%D0%B9%20%D0%B7%D0%BE%D0%BD%D1%8B%20%D0%91%D0%B0%D0%B9%D0%BA%D0%B0%D0%BB%D1%8C%D1%81%D0%BA%D0%BE%D0%B9%20%D0%BF%D1%80%D0%B8%D1%80%D0%BE%D0%B4%D0%BD%D0%BE%D0%B9%20%D1%82%D0%B5%D1%80%D1%80%D0%B8%D1%82%D0%BE%D1%80%D0%B8%D0%B8:0)

# Типовые правила расчета предельно допустимой рекреационной емкости особо охраняемых природных территорий регионального и местного значения при осуществлении туризма

### 1. Настоящие Типовые правила устанавливают порядок расчета предельно допустимой рекреационной емкости особо охраняемых природных территорий регионального и местного значения в рамках организации на их территориях туризма.

Расчет предельно допустимой рекреационной емкости особо охраняемых природных территорий регионального и местного значения, расположенных в границах центральной экологической зоны Байкальской природной территории, в рамках организации на их территориях туризма исполнительными органами Республики Бурятия и Иркутской области осуществляется с учетом требований статьи 12 Федерального закона "Об охране озера Байкал".

### 2. В целях настоящих Типовых правил используются следующие понятия:

- **"автономный многодневный туристский маршрут"** - туристский маршрут, длящийся более одного дня, при котором туристы не используют места размещения (туристские базы, гостиницы, кемпинги) и пункты питания туристов;

- **"базовая рекреационная емкость"** - максимальное количество человек, которые могут физически находиться на особо охраняемой природной территории или в отдельной части (на туристском объекте) в единицу времени;

- **"лимитирующий фактор развития туризма"** - фактор, ограничивающий возможности развития туризма на особо охраняемой природной территории или в отдельной части (на туристском объекте) ввиду несовместимости туризма и обеспечения сохранности уникальных и типичных природных комплексов и объектов, объектов растительного и животного мира, естественных экологических систем, биоразнообразия в целях поддержания их в естественном состоянии, а также невозможности оказания посетителям услуг, соответствующих договору или обычно предъявляемым к ним требованиям, и снижения негативного воздействия на местную социокультурную среду;

- **"линейный туристский объект (туристский маршрут)"** - путь следования туристов, расположенный на особо охраняемой природной территории, характеризующийся линейным характером воздействия на окружающую среду вдоль пути следования;

- **"многодневный туристский маршрут"** - туристский маршрут, прохождение которого длится более одного дня, при котором туристы размещаются на ночевку в местах размещения;

- **"однодневный туристский маршрут"** - туристский маршрут, прохождение которого совершается в течение дня, без использования мест размещения;

- **"площадной туристский объект"** - туристский объект, занимающий определенный земельный участок или участок акватории, имеющий свои границы, характеризующийся свободным перемещением на нем посетителей и площадным (обширным, по всей площади объекта или его части) характером воздействия его на окружающую среду (стоянки, смотровые площадки, музеи, визит-центры, иные здания и сооружения для организации обслуживания туристов);

- **"потенциальная рекреационная емкость"** - максимальное количество человек, которые могут находиться на особо охраняемой природной территории, в ее отдельной части или на туристском объекте в единицу времени без деградации природных комплексов и объектов, объектов растительного и животного мира, естественных экологических систем, с учетом факторов экологического, социального, социокультурного и социально-экономического характера;

- **"предельно допустимая рекреационная емкость"** - максимальное количество посетителей, которые могут посетить в качестве туриста особо охраняемую природную территорию либо ее отдельные части в единицу времени без деградации природных комплексов и объектов, объектов растительного и животного мира, естественных экологических систем;

- **"туристские объекты"** - части особо охраняемой природной территории, включающие природные комплексы и объекты, историко-культурные объекты, инфраструктурные объекты (музеи, визит-центры, иные здания и сооружения для организации обслуживания туристов), привлекающие туристов и используемые для осуществления туризма.

### 3. Предельно допустимая рекреационная емкость определяется для особо охраняемой природной территории либо ее отдельных частей (туристских объектов).

### 4. Предельно допустимая рекреационная емкость устанавливается исполнительными органами субъектов Российской Федерации в отношении находящихся в их ведении особо охраняемых природных территорий регионального значения и органами местного самоуправления в отношении находящихся в их ведении особо охраняемых природных территорий местного значения.

### 5. Расчет предельно допустимой рекреационной емкости особо охраняемой природной территории, ее отдельной части осуществляется при выявлении изменения состояния туристских объектов, но не реже одного раза в 5 лет.

### 6. Предельно допустимая рекреационная емкость рассчитывается для особо охраняемой природной территории в целом, а также для ее отдельных частей (туристских объектов) и выражается в целочисленных значениях, человек в единицу времени.



In [1]:
# conda activate //anaconda3/envs/condageoenv
import math
import osmnx as ox
import folium
import matplotlib.pyplot as plt

*Задайте область интереса. В вашем случае это Камчатка и конкретно объекты, относящиеся к особо охраняемым территориям. Вы можете загрузить данные с определенными тегами:*

In [2]:
# Установите URL для Overpass API
ox.settings.overpass_url = "http://overpass-api.de/api/"

# Укажите название области
place_name = "Камчатский край, Россия"

# Получите границы особо охраняемых природных территорий
oopt = ox.geocode_to_gdf(place_name)
map_center = [oopt.geometry.centroid.y, oopt.geometry.centroid.x]  # Центр Камчатки по умолчанию

# Исправление вызова функции для получения геометрий
oopt_polygons = ox.features_from_place(place_name, tags={'boundary': 'protected_area', 'protect_class': '*'})

# Загружаем точки, помеченные как памятники природы
points = ox.features_from_place(place_name, tags={
    'leisure': 'nature_reserve', # этот тег позволяет извлекать объекты, помеченные как зоны отдыха, которые являются природными заповедниками.
    'tourism': 'nature_reserve', # этот тег позволяет находить объекты, связанные с туризмом, которые также классифицированы как природные заповедники.
    'natural': ['peak', 'volcano'], # здесь выбираются объекты, относящиеся к природным особенностям, а именно горные вершины и вулканы.
    'historic': 'monument' # этот тег извлекает объекты, которые имеют историческое значение и классифицируются как памятники
})

# Фильтруем только точки
points = points[points.geometry.type == 'Point']

# Создайте карту с фокусом на загруженную область
my_map = folium.Map(location=map_center, zoom_start=6)

# Слои с возможностью отключения
feature_group_oopt = folium.FeatureGroup(name="Особо охраняемые природные территории")
feature_group_points = folium.FeatureGroup(name="Памятники природы", show=False)

# Добавьте границы особо охраняемых территорий на карту
for index, row in oopt_polygons.iterrows():
    popup_content = f"<b>Название:</b> {row.get('name', 'Без названия')}<br>" \
                    f"<b>Класс защиты:</b> {row.get('protect_class', 'Не указан')}<br>" \
                    f"<b>Оффициальное название:</b> {row.get('official_name', 'Не указан')}<br>" \
                    f"<b>Дата основания:</b> {row.get('start_date', 'Не указан')}<br>" \
                    f"<b>Веб-сайт:</b> {row.get('website', 'Не указан')}<br>"

    folium.GeoJson(
        row['geometry'],
        popup=folium.Popup(popup_content, max_width=300),
        tooltip=row.get('name', 'ООПТ'),
    ).add_to(feature_group_oopt)

# Добавляем маркеры памятников природы
for index, row in points.iterrows():
    folium.Marker(
        location=[row['geometry'].y, row['geometry'].x],
        popup=row.get('name', 'Без названия')
    ).add_to(feature_group_points)

# Добавляем группы на карту
feature_group_oopt.add_to(my_map)
feature_group_points.add_to(my_map)

folium.LayerControl().add_to(my_map)  # Кнопка управления слоями

# Отобразите карту
my_map

/var/folders/b9/wz3thh3n6176vcms9jxkj4wr0000gn/T/ipykernel_53539/1511172037.py:9: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  map_center = [oopt.geometry.centroid.y, oopt.geometry.centroid.x]  # Центр Камчатки по умолчанию
/anaconda3/envs/condageoenv/lib/python3.10/site-packages/osmnx/_overpass.py:254: UserWarning: This area is 417 times your configured Overpass max query area size. It will automatically be divided up into multiple sub-queries accordingly. This may take a long time.
  multi_poly_proj = utils_geo._consolidate_subdivide_geometry(poly_proj)
/anaconda3/envs/condageoenv/lib/python3.10/site-packages/pandas/core/dtypes/cast.py:1641: DeprecationWarning: np.find_common_type is deprecated.  Please use `np.result_type` or `np.promote_types`.
See https://numpy.org/devdocs/release/1.25.0-notes.html and the docs for more information.  (Deprec

### 7. Предельно допустимая рекреационная емкость особо охраняемой природной территории (RСCоопт) рассчитывается с помощью функции:

In [2]:
def calculation_RCCoopt(features: list[int], RCCqm: list[int]) -> tuple[dict, int]:
    '''
    RCCqm - предельно допустимая рекреационная емкость туристского объекта m, человек в единицу времени;
    features - список туристских объектов (0...m-1).
    '''

    # Создаем словарь с ключами из списка номеров (или названий) объектов и рассчитанными значениями RCCq
    dict_features_RCCqm = {feature: rccq for feature, rccq in zip(features, RCCqm)}
    
    result = int(sum(dict_features_RCCqm.values()))
    return dict_features_RCCqm, result

### 8. Предельно допустимая рекреационная емкость туристского объекта (RCCq) рассчитывается с помощью функции:

In [3]:
def calculation_RCCq(PCCq: list[int], MC: list[float]) -> list[int]:
    '''
    PCCq - потенциальная рекреационная емкость туристского объекта, человек в единицу времени;
    MC - коэффициент управленческой емкости, долей от единицы.
    '''
    
    result = []
    
    for pccq, mc in zip(PCCq, MC):
        result.append(int(pccq * mc))
    
    return result

### 9. Потенциальная рекреационная емкость рассчитывается для особо охраняемой природной территории в целом, а также для ее отдельных частей (туристских объектов) и выражается в целочисленных значениях, человек в единицу времени.

### 10. Потенциальная рекреационная емкость туристского объекта (PCCq) рассчитывается c помощью функции:

In [4]:
def calculation_PCCq(BCCq: list[int], Cfn: list[list[int]]) -> list[int]:
    '''
    BCCq - базовая рекреационная емкость туристского объекта, выраженная в целочисленном значении, человек в единицу времени;
    Cfn - поправочные коэффициенты, которые учитывают определенные для туристских объектов лимитирующие факторы развития туризма (экологического, социального и социокультурного характера) и установленные режимы использования туристских объектов;
    n - количество поправочных коэффициентов. 
    '''

    # Перемножаем элементы вложенных в список Cf списков
    Cf_multiplied_sublists = [math.prod(sublist) for sublist in Cfn]
    
    result = []
    
    for bccq, cf in zip(BCCq, Cf_multiplied_sublists):
        result.append(int(bccq * cf))

    return result

### 11. Базовая рекреационная емкость туристских объектов (BCC q) для площадных и линейных туристских объектов (туристских маршрутов) рассчитывается по приведенным формулам и выражается в целочисленных значениях, человек в единицу времени.

### 12. Базовая рекреационная емкость для ПЛОЩАДНЫХ туристских объектов (BCC qs) рассчитываетсяс помощь функции:

In [5]:
def calculation_BBCqs(A: list[float], Au: list[float], Rf: list[float], t: int) -> list[int]:
    '''
    A - площадь туристского объекта, на которой осуществляется туризм, кв. метров;
    Au - площадь туристского объекта, необходимая для одного посетителя при осуществлении туризма (кв. метров), определяемая в соответствии с порядком, указанным в пункте 18 настоящих Правил;
    Rf - коэффициент возвращения, отражающий возможное количество посещений туристского объекта одним и тем же туристом в день;
    t - количество дней в рассматриваемую единицу времени (месяц, сезон, год и др.), единиц.
    '''
    
    result = []
    
    for area, area_per_visitor, return_factor in zip(A, Au, Rf):
        bbcq = (area / area_per_visitor) * return_factor * t
        result.append(int(bbcq)) 

    return result


### 13. Коэффициент возвращения (Rf) рассчитывается по формуле:

In [6]:
def calculation_Rf(T: list[int], Td: list[int]) -> list[float]:
    '''
    Т - количество часов в сутки, когда туристский объект доступен для посещения, часов;
    Td - среднее время пребывания посетителя на туристском объекте, часов.
    '''

    result = []
    
    for times, mean_times in zip(T, Td):
        rf = times / mean_times 
        result.append(float(rf)) 

    return result

### 14. Базовая рекреационная емкость для ЛИНЕЙНЫХ: однодневных туристских маршрутов и многодневных туристских маршрутов с небольшой протяженностью или несколькими входами без ограничения времени посещения туристского маршрута (BCC qp1) рассчитывается по формуле:

In [7]:
def calculation_BBCqp1(
        DTp: list[list[float]], 
        DGp: list[list[float]], 
        Ts: float,
        Tdp: list[list[float]],
        t: int,
        GS: float
        # tp: list[int]
        ) -> list[float]:
    '''
    DTp - протяженность однодневного туристского маршрута или однодневного участка p многодневного туристского маршрута в дневной переход, км;
    DGp - оптимальное расстояние между группами на участке p туристского маршрута, км;
    Ts - длина светового дня или количество времени, когда туристский маршрут доступен для посетителей, часов;
    Tdp - среднее время прохождения участка туристского маршрута p с учетом остановок, часов;
    GS - среднее количество человек в группе (включая сопровождающих), человек;
    p - порядковый номер однодневного участка туристского маршрута (1, 2, ... p);
    t - количество дней в рассматриваемую единицу времени (месяц, сезон, год и др.), единиц;
    tp - количество дней пребывания посетителей на туристском маршруте, единиц.
    '''
    
    result = []
    p = 0 # Счетчик однодневных участков в многодневном маршруте

    # Обработка каждого вложенного списка
    for dtp_list, dgp_list, tdp_list in zip(DTp, DGp, Tdp):
        sum_for_current = 0 # хранилище сумм
        # Рассчитаем результат в отдельных вложенных списках
        for dtp, dgp, tdp in zip(dtp_list, dgp_list, tdp_list):
            # считаем результирующее значение и суммируем со следующим рассчитанным
            # в рассматриваемом вложенном списке 
            sum_for_current += (dtp / dgp) * (Ts / tdp)
        p += 1 

        tp = len(DTp[p - 1])
        result.append(round(sum_for_current * GS * (t / tp), 1)) 

    return result

# Пример входных данных (трех маршрутов 1 и 3 многодневные, а 2 - однодневный) 
# посути это список со вложенными списками разной длины
DTp = [[20, 25], [30], [30, 20, 50]]
DGp = [[5, 10], [10], [10, 5, 20]]
Ts = 8.0
Tdp = [[4, 3], [3], [3, 4, 3]]
t = 30
GS = 3.1

# Вызов функции
out = calculation_BBCqp1(
        DTp, DGp, Ts, Tdp, t, GS
        )
print(out)

[682.0, 744.0, 702.7]


### 15. Базовая рекреационная емкость для однодневных и многодневных туристских маршрутов, время доступности которых строго фиксировано (BCC qp2) (например, в случае закрытия для посетителей входа и выхода с туристского маршрута или в целом с участка особо охраняемой природной территории в четко установленные часы), рассчитывается по формуле:

In [8]:
def calculation_BBCqp2(
        gp: list[list[int]],
        t: int,
        GS: float
        ) -> list[float]:
    '''
    gp - максимальное количество групп, которые могут пройти в сутки по 
    однодневному участку p туристского маршрута до его закрытия или до окончания светового дня, единиц;
    GS - среднее количество человек в группе (включая сопровождающих), человек;
    p - порядковый номер однодневного участка туристского маршрута (1, 2, ... p);
    t - количество дней в рассматриваемую единицу времени (месяц, сезон, год и др.), единиц;
    tp - количество дней пребывания посетителей на туристском маршруте, единиц.
    '''
    
    result = []
    p = 0 # Счетчик однодневных участков в многодневном маршруте

    # Обработка каждого вложенного списка
    for gp_list in gp:
        sum_for_current = 0 # хранилище сумм
        # Рассчитаем результат в отдельных вложенных списках
        for gp_value in gp_list:
            # считаем результирующее значение и суммируем со следующим рассчитанным
            # в рассматриваемом вложенном списке 
            sum_for_current += gp_value * GS
        p += 1 

        tp = len(gp[p - 1])
        result.append(round(sum_for_current * (t / tp), 1)) 

    return result

gp = [[5, 6], [7], [7, 5, 12]]
t = 30
GS = 3.0
# Вызов функции
out = calculation_BBCqp2(
        gp, t, GS
        )
print(out)

[495.0, 630.0, 720.0]


### 16. Максимальное количество групп, которые могут пройти в сутки по однодневному участку туристского маршрута до его закрытия или до окончания светового дня (gp), выражается целочисленным значением (единиц) и определяется по формуле:

In [9]:
def calculation_gp(
        DGp: list[list[float]], 
        Ts: float,
        Tdp: list[list[float]],
        vp: list[float]
        ) -> list[list[int]]:
    '''
    DGp - оптимальное расстояние между группами на участке p туристского маршрута, км;
    Ts - длина светового дня или количество времени, когда туристский маршрут доступен для посетителей, часов;
    Tdp - среднее время прохождения участка туристского маршрута p с учетом остановок, часов;
    vp - средняя скорость передвижения по однодневному участку p туристского маршрута с учетом остановок, км в час.
    '''
    
    result = []

    for dgp_list, tdp_list, vp_list in zip(DGp, Tdp, vp):
        subresult = []
        for dgp, tdp, vp_value in zip(dgp_list, tdp_list, vp_list):
            calculated_value = int(1 + ((vp_value * (Ts - tdp)) / dgp))
            subresult.append(calculated_value)
        result.append(subresult)

    return result

DGp = [[5, 10], [10], [10, 5, 20]]
Ts = 8.0
Tdp = [[4, 3], [3], [3, 4, 3]]
vp = [[4.0, 4.0], [5.0], [4.0, 4.0, 6.0]]

# Вызов функции
out = calculation_gp(
        DGp, Ts, Tdp, vp
        )
print(out)

[[4, 3], [3], [3, 4, 2]]


### 17. Базовая рекреационная емкость для автономных многодневных туристских маршрутов (BBCqp3) рассчитывается по формуле:

In [10]:
def calculation_BBCqp3(
        gp: list[list[int]],
        t: int,
        GS: float
        ) -> list[list[int]]:
    '''
    gp - максимальное количество групп, которые могут пройти в сутки по однодневному участку 
    туристского маршрута до его закрытия или до окончания светового дня (gp), единиц;
    gpmin - минимальное из рассчитанных для однодневных участков туристского маршрута значений 
    максимального количества групп, которые могут пройти в сутки по однодневному участку p 
    туристского маршрута до его закрытия или до окончания светового дня, единиц;
    GS - среднее количество человек в группе (включая сопровождающих), человек;
    p - порядковый номер однодневного участка туристского маршрута (1, 2, ... p);
    t - количество дней в рассматриваемую единицу времени (месяц, сезон, год и др.), единиц.
    '''
    
    result = []

    for gpmin_list in gp:
        gp_min = min(gpmin_list)  # Находим минимальное значение
        calculated_value = round(gp_min * GS * t, 1)
        result.append(calculated_value)

    return result

gp = calculation_gp(
        DGp, Ts, Tdp, vp
        )
t = 30
GS = 3.0

# Вызов функции
out = calculation_BBCqp3(
        gp, t, GS
        )
print(out) 

[270.0, 270.0, 180.0]


In [11]:
# Определяем список туристических объектов (!!! Все три объекта ПЛОЩАДНЫЕ !!!)
tourist_features = ['Объект 1', 'Объект 2', 'Объект 3']

# Определяем коэффициенты возвращения туристов на объекты
Rf_list = calculation_Rf([8, 7, 24], [5, 7, 12])
print(f'Коэффициенты возвращения туристов на объекты: {Rf_list}, единиц времени')

# Определяем базовую рекреационную емкость для туристских объектов ЕСЛИ ВСЕ ОНИ ПЛОЩАДНЫЕ 
# !!! пока логика разделения на линейные и площадные объекты на реализована и 
# все объекты должна задаваться одним типом: линейным или площадным !!!
BBCqs_list = calculation_BBCqs([100.2, 50.0, 25.1], 
                                [30.5, 12.0, 25.0], 
                                Rf_list, 
                                30)
print(f'Базовые рекреационные емкости: {BBCqs_list}, человек в единицу времени')

# Определяем потенциальную рекреационную емкость для каждого объекта
PCCq_list = calculation_PCCq(BBCqs_list, [[0.4, 0.5], [0.5, 2, 0.3], [0.8]])   
print(f'Потенциальные рекркационные емкости: {PCCq_list}, человек в единицу времени')

# Определяем коэффициенты управленческой емкости для каждого объекта
MC_list = [0.8, 0.5, 0.75]

# Вычисляем список придельно допустимых туристских емкостей объектов
RCCq_list = calculation_RCCq(PCCq_list, MC_list)
print(f'Придельно допустимые туристские емкости: {RCCq_list}, человек в единицу времени')

# Пусть у нас m туристических объекта c соответствующими предельно допустимыми рекреационными емкостями, 
# тогда предельно допустимая рекреационная емкость особо охраняемой природной территории RCCоопт будет равна   
RCCoopt_int = calculation_RCCoopt(tourist_features, RCCq_list)
print(f'Предельно допустимая рекреационная емкость ООПТ: {RCCoopt_int}, человек в единицу времени')

Коэффициенты возвращения туристов на объекты: [1.6, 1.0, 2.0], единиц времени
Базовые рекреационные емкости: [157, 125, 60], человек в единицу времени
Потенциальные рекркационные емкости: [31, 37, 48], человек в единицу времени
Придельно допустимые туристские емкости: [24, 18, 36], человек в единицу времени
Предельно допустимая рекреационная емкость ООПТ: ({'Объект 1': 24, 'Объект 2': 18, 'Объект 3': 36}, 78), человек в единицу времени


In [12]:
# Определяем список туристических объектов (!!! Все три объекта ЛИНЕЙНЫЕ !!!)
tourist_features = ['Объект 1', 'Объект 2', 'Объект 3']

# Определяем коэффициенты возвращения туристов на объекты
Rf_list = calculation_Rf([8, 7, 24], [5, 7, 12])
print(f'Коэффициенты возвращения туристов на объекты: {Rf_list}, единиц времени')

# Определяем базовую рекреационную емкость для туристских объектов для однодневных туристских маршрутов 
# и многодневных туристских маршрутов с небольшой протяженностью или несколькими входами 
# без ограничения времени посещения туристского маршрута, т.е. ЕСЛИ ВСЕ ОНИ ЛИНЕЙНЫЕ 
# !!! пока логика разделения на линейные и площадные объекты на реализована и 
# все объекты должна задаваться одним типом: линейным или площадным !!!
BBCqp1_list = calculation_BBCqp1([[20, 25], [30], [30, 20, 50]], 
                                [[5, 10], [10], [10, 5, 20]], 
                                8.0, 
                                [[4, 3], [3], [3, 4, 3]],
                                30,
                                3.1
                                )
print(f'Базовые рекреационные емкости линейных объектов BBCqp1: {BBCqp1_list}, человек в единицу времени')

print('- ' * 18)
# Максимальное количество групп на маршрутах
gp = calculation_gp([[5, 10], [10], [10, 5, 20]],
                    8.0,
                    [[4, 3], [3], [3, 4, 3]],
                    vp = [[4.0, 4.0], [5.0], [4.0, 4.0, 6.0]]
                    )
print(f'Максимальное количество групп на маршрутах: {gp}, единиц')

# Базовая рекреационная емкость для однодневных и многодневных туристских маршрутов, 
# время доступности которых строго фиксировано (BCC qp2) (например, в случае закрытия для посетителей 
# входа и выхода с туристского маршрута или в целом с участка особо охраняемой природной территории 
# в четко установленные часы), рассчитывается по формуле:
BBCqp2_list = calculation_BBCqp2(
        [[5, 6], [7], [7, 5, 12]], 
         t,
         GS
        )
print(f'Базовые рекреационные емкости линейных объектов BBCqp2: {BBCqp2_list}, человек в единицу времени')
print('- ' * 18)

# Базовая рекреационная емкость для автономных многодневных туристских маршрутов (BBCqp3) рассчитывается по формуле:
BBCqp3_list = calculation_BBCqp3(
        gp,
        t,
        GS
        )
print(f'Базовые рекреационные емкости линейных объектов BBCqp3: {BBCqp3_list}, человек в единицу времени')
print('- ' * 18)

# Определяем потенциальную рекреационную емкость для каждого объекта
PCCq_list = calculation_PCCq(BBCqp1_list, [[0.4, 0.5], [0.5, 2, 0.3], [0.8]])   
print(f'Потенциальные рекркационные емкости: {PCCq_list}, человек в единицу времени')

# Определяем коэффициенты управленческой емкости для каждого объекта
MC_list = [0.8, 0.5, 0.75]

# Вычисляем список придельно допустимых туристских емкостей объектов
RCCq_list = calculation_RCCq(PCCq_list, MC_list)
print(f'Придельно допустимые туристские емкости: {RCCq_list}, человек в единицу времени')

# Пусть у нас m туристических объекта c соответствующими предельно допустимыми рекреационными емкостями, 
# тогда предельно допустимая рекреационная емкость особо охраняемой природной территории RCCоопт будет равна   
RCCoopt_int = calculation_RCCoopt(tourist_features, RCCq_list)
print(f'Предельно допустимая рекреационная емкость ООПТ: {RCCoopt_int}, человек в единицу времени')

Коэффициенты возвращения туристов на объекты: [1.6, 1.0, 2.0], единиц времени
Базовые рекреационные емкости линейных объектов BBCqp1: [682.0, 744.0, 702.7], человек в единицу времени
- - - - - - - - - - - - - - - - - - 
Максимальное количество групп на маршрутах: [[4, 3], [3], [3, 4, 2]], единиц
Базовые рекреационные емкости линейных объектов BBCqp2: [495.0, 630.0, 720.0], человек в единицу времени
- - - - - - - - - - - - - - - - - - 
Базовые рекреационные емкости линейных объектов BBCqp3: [270.0, 270.0, 180.0], человек в единицу времени
- - - - - - - - - - - - - - - - - - 
Потенциальные рекркационные емкости: [136, 223, 562], человек в единицу времени
Придельно допустимые туристские емкости: [108, 111, 421], человек в единицу времени
Предельно допустимая рекреационная емкость ООПТ: ({'Объект 1': 108, 'Объект 2': 111, 'Объект 3': 421}, 640), человек в единицу времени
